In [18]:
# bayesian personalized ranking
# based on: http://ethen8181.github.io/machine-learning/recsys/4_bpr.html

# loading packages
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import ceil
from tqdm import trange
from itertools import product, islice
from collections import namedtuple
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.preprocessing import normalize
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix, dok_matrix
from typing import Iterable, Any, Dict
import pickle


from matplotlib.pyplot import rcParams
import seaborn as sb

In [19]:
%matplotlib inline
rcParams['figure.figsize'] = 8, 4
sb.set_style('whitegrid')

## Data Preparation

In [ ]:
# location for the datasets

address2 = 'CSV/cdnelastic_VoD.csv'
address = 'CSV/videos.csv'

In [50]:
# loading the datasets

# allcols = ['timestamp','statuscode','contentlength','host','timefirstbyte','timetoserv','hit','contenttype',
#            'cachecontrol','cachename','popname','method','protocol','path','uid','sid','livechannel',
#            'contentpackage','assetnumber','maxage','coordinates','devicebrand','devicefamily','devicemodel',
#            'osfamily','uafamily','uamajor','manifest','fragment']
# somecols = ['@timestamp','statuscode','contentlength','host','timetoserv','hit','contenttype',
#               'protocol','uid','sid','livechannel','contentpackage','assetnumber','coordinates',
#               'uafamily']
# missing_values = ['n/a','na','--','NaN','NA','-']
# # cdnset2 = pd.read_csv(address2,
# #                      header=0,
# #                      nrows=1000000,
# #                      parse_dates=[0],
# #                      comment='#',
# #                      names=somecols,
# #                      na_values=missing_values
# #                     )
# # print(cdnset2.info())
# cdnset2 = pd.read_csv(address2,  na_values=missing_values)

file_path = 'CSV/ml-100k/u.data'
# we will not be using the timestamp column
names = ['user_id', 'item_id', 'rating', 'timestamp']
cdnset2 = pd.read_csv(file_path, sep = '\t', names = names)
print('data dimension: \n', cdnset2.shape)
cdnset2.head()

# cdnset2.head(2)

data dimension: 
 (100000, 4)


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [ ]:
# cdnset = cdnset2[['timestamp', 'uid', 'livechannel']]

In [ ]:
cdnset = cdnset2[['@timestamp', 'uid', 'contentpackage']]
# cdnset = pd.read_csv(address,  na_values=missing_values)

In [ ]:
# cdnset.info()

In [ ]:
# cdnset.head(2)

In [ ]:
cdnset['@timestamp'] = pd.to_datetime(cdnset['@timestamp'])

/tmp/ipykernel_211/1231525476.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdnset['@timestamp'] = pd.to_datetime(cdnset['@timestamp'])


In [ ]:
# cdnset.head(2)

In [ ]:
# cdnset.info()

In [ ]:
# removing missing values for user id

cdnset3 = cdnset.dropna(subset=['uid'])

In [ ]:
# cdnset3.info()

In [ ]:
df_cdn = cdnset3.groupby(['uid', 'contentpackage'], as_index=False).count()
df_cdn = df_cdn.astype(int)
# df_cdn.head(2)

In [ ]:
'''
find a different way to engineer the ratings, not using frequencies.
this is different with liveTV

Engineer ratings in a binary way, e.g., yes/no, if requested then yes, otherwise no (binary rating)
Research if it makes sense.

!!! focus on this issue of sparsity

The same user could have uniquely different ids across different hosts
need to correspondence among
Have to unify (across all hosts and all days) different attibutes of different ids(users, livechannels, etc.), across different hosts at least to curb the problem of sparsity
.

Looking at the data at the host level for different time windows, very low priority to uid
Host based analytics with time. (host/time vs content)

'''
df_cdn.rename(columns={'@timestamp':'requests'}, inplace=True)
df_cdn.head(2)

,uid,contentpackage,requests
0,24,3528,78
1,59,918,1


In [ ]:
df_cdn.shape

(16109, 3)

In [ ]:
# df_cdn.info()

In [ ]:
print('unique uid: ',len(df_cdn.uid.unique()))
print('unique vod: ',len(df_cdn.contentpackage.unique()))

unique uid:  3670
unique vod:  3422


### Will start by constructing a user-item matrix

In [ ]:
# creating the matrix

# ratings1 = df_cdn.pivot(index='uid', columns='contentpackage').fillna(0, downcast='infer')
# print(ratings1.shape)

In [ ]:
# ratings = np.array(ratings1)
# print(ratings.shape)
# print(ratings)

In [21]:
## better way to create the matrix

def create_matrix(data, users_col, items_col, ratings_col, threshold=None):
    '''
    creating a user-item interaction matrix
    '''
    if threshold is not None:
        data = data[data[ratings_col] >= threshold]
        data[ratings_col] = 1
        
    for col in (items_col, users_col, ratings_col):
        data[col] = data[col].astype('category')
        
    ratings = csr_matrix((data[ratings_col], (data[users_col].cat.codes, data[items_col].cat.codes)))
    ratings.eliminate_zeros()
    return ratings, data

In [51]:
items_col = 'item_id' # 'contentpackage'
users_col = 'user_id' # 'uid'
ratings_col = 'rating' # 'requests'
threshold = 3
# df = df_cdn

X, df = create_matrix(cdnset2, users_col, items_col, ratings_col, threshold)
X

/tmp/ipykernel_211/2420268622.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[ratings_col] = 1
/tmp/ipykernel_211/2420268622.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = data[col].astype('category')
/tmp/ipykernel_211/2420268622.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

<943x1574 sparse matrix of type '<class 'numpy.int64'>'
	with 82520 stored elements in Compressed Sparse Row format>

In [ ]:
# snapshot of users rating how many items
# df_cdn.groupby('uid').count()['contentpackage']

# shows at least one vod is rated by a particular user
# thus for random split we will use this for testing algorithm

# '''
# number of users who requested how many movies.
# '''

# plt.hist(np.sum(ratings != 0, axis = 1), range=[0, 30],histtype = 'stepfilled', bins = 90, label = '# of ratings')
# plt.axvline(x = 1, color = 'black', linestyle = '--')
# plt.legend(loc = "upper right")
# plt.show()

In [ ]:
# checking for non-zero elements in the matrix
# if dataset too sparse (sparsity below 1%)

# matrix_size = np.prod(ratings.shape)
# interaction = np.flatnonzero(ratings).shape[0]
# sparsity = 100 * (interaction / matrix_size)

# print('dimension: ', ratings.shape)
# print('sparsity: {:.1f}%'.format(sparsity))
# ratings

In [23]:
# randomly splitting the dataset into train and test sets

def create_train_test(ratings, test_size = 0.2, seed = 1234):
    
    assert test_size < 1.0 and test_size > 0.0
    
    
    train = ratings.copy().todok()
    test = dok_matrix(train.shape)
    
    # randomly chosen interactions
    # masking interactions in training and assigning them to the test set
    rstate = np.random.RandomState(seed)
    for u in range(ratings.shape[0]):
        split_index = ratings[u].indices
        n_splits = ceil(test_size * split_index.shape[0])
        test_index = rstate.choice(split_index, size = n_splits, replace = False)
        test[u, test_index] = ratings[u, test_index]
        train[u, test_index] = 0
        
    train, test = train.tocsr(), test.tocsr()
    return train, test

In [52]:
X_train, X_test = create_train_test(X, test_size = 0.2, seed = 1234)
X_train

<943x1574 sparse matrix of type '<class 'numpy.int64'>'
	with 65641 stored elements in Compressed Sparse Row format>

In [53]:
X_test

<943x1574 sparse matrix of type '<class 'numpy.float64'>'
	with 16879 stored elements in Compressed Sparse Row format>

## BPR Matrix Factorization

In [28]:
# creating the matrix factorization (MF) class for modeling purpose

class BPRMFClass:
    """
    Bayesian Personalized Ranking (BPR) for implicit feedback data

    Parameters
    ----------
    learning_rate (alpha) : float, default 0.01
        learning rate for gradient descent

    n_factors (hidden) : int, default 20
        Number/dimension of user and item latent factors

    n_iters : int, default 15
        Number of iterations to train the algorithm
        
    batch_size : int, default 1000
        batch size for batch gradient descent, the original paper
        uses stochastic gradient descent (i.e., batch size of 1),
        but this can make the training unstable (very sensitive to
        learning rate)

    reg (beta) : int, default 0.01
        Regularization term for the user and item latent factors

    seed : int, default 1234
        Seed for the randomly initialized user, item latent factors

    verbose : bool, default True
        Whether to print progress bar while training

    Attributes
    ----------
    user_factors : 2d ndarray, shape [n_users, n_factors]
        User latent factors learnt

    item_factors : 2d ndarray, shape [n_items, n_factors]
        Item latent factors learnt

    References
    ----------
    S. Rendle, C. Freudenthaler, Z. Gantner, L. Schmidt-Thieme 
    Bayesian Personalized Ranking from Implicit Feedback
    - https://arxiv.org/abs/1205.2618
    """
    
    def __init__(self, alpha = 0.01, hidden = 15, n_iters = 10, batch_size = 1000, reg = 0.01, seed = 1234, verbose = True):
        self.reg = reg
        self.seed = seed
        self.verbose = verbose
        self.n_iters = n_iters
        self.hidden = hidden
        self.batch_size = batch_size
        self.alpha = alpha
        
        # to avoid re-computation at predict
        self._prediction = None
        
    def fit(self, ratings):
        '''
        ratings: scipy sparse csr_matrix; user-item interactions
        '''
        indptr = ratings.indptr
        indices = ratings.indices
        n_users, n_items = ratings.shape
        
        # ensure batch size to be smaller than total number of users to
        # avoid sampling with replacement
        batch_size = self.batch_size
        if n_users < batch_size:
            batch_size = n_users
            sys.stderr.write('size greater than user count, update batch size of {}\n'.format(n_users))
            
        batch_iters = n_users // batch_size
        
        #initialize random weights
        rstate = np.random.RandomState(self.seed)
        self.user_factors = rstate.normal(size = (n_users, self.hidden))
        self.item_factors = rstate.normal(size = (n_items, self.hidden))
        
        # tracking progress with verbose, if it is on
        loop = range(self.n_iters)
        if self.verbose:
            loop = trange(self.n_iters, desc = self.__class__.__name__)
            
        for _ in loop:
            for _ in range(batch_iters):
                sampled = self._sample(n_users, n_items, indices, indptr)
                #print('sampled:',sampled)
                sampled_users, sampled_pos_items, sampled_neg_items = sampled
                self._update(sampled_users, sampled_pos_items, sampled_neg_items)
                
        return self
    
    def _sample(self, n_users, n_items, indices, indptr):
        '''sample batches of random triplets u, i, j'''
        sampled_pos_items = np.zeros(self.batch_size, dtype = int)
        sampled_neg_items = np.zeros(self.batch_size, dtype = int)
        sampled_users = np.random.choice(n_users, size = self.batch_size, replace = False)
        #print('sampled users:',sampled_users)
        
        for idx, user in enumerate(sampled_users):
            pos_items = indices[indptr[user]:indptr[user + 1]]
            #print('positive itemsss:',pos_items)
            pos_item = np.random.choice(pos_items)
            #print('positive item:',pos_item)
            neg_item = np.random.choice(n_items)
            #print('negative item:',neg_item)
            while neg_item in pos_items:
                neg_item = np.random.choice(n_items)
                
            sampled_pos_items[idx] = pos_item
            sampled_neg_items[idx] = neg_item
            
        return sampled_users, sampled_pos_items, sampled_neg_items
    
    def _update(self, u, i, j):
        '''
        updating according to bootstrapped user u, positive item i and
        negative item j
        '''
        user_u = self.user_factors[u]
        item_i = self.item_factors[i]
        item_j = self.item_factors[j]
        
        # compute difference between the score of positive items and negative items
        r_uij = np.sum(user_u * (item_i - item_j), axis = 1)
        sigmoid = np.exp(-r_uij) / (1.0 + np.exp(-r_uij))
        
        # repeat the 1 dimension sigmoid hidden_factors times so that
        # dimensions will match when doing the update
        sigmoid_tiled = np.tile(sigmoid, (self.hidden, 1)).T
        
        # update using gradient descent (optimization)
        grad_u = sigmoid_tiled * (item_j - item_i) + self.reg * user_u
        grad_i = sigmoid_tiled * -user_u + self.reg * item_i
        grad_j = sigmoid_tiled * user_u + self.reg * item_j
        self.user_factors[u] -= self.alpha * grad_u
        self.item_factors[i] -= self.alpha * grad_i
        self.item_factors[j] -= self.alpha * grad_j
        return self
    
    def predict(self):
        '''
        predicted ratings for every user and items
        '''
        if self._prediction is None:
            self._prediction = self.user_factors.dot(self.item_factors.T)
            
        return self._prediction
    
    def _predict_user(self, user):
        '''return predicted ratings for specified user,
        mainly used in computing evaluation metric
        '''
        user_pred = self.user_factors[user].dot(self.item_factors.T)
        return user_pred
    
    def recommend(self, ratings, N = 5):
        '''
        return top N ranked items for given user id
        '''
        n_users = ratings.shape[0]
        recommendation = np.zeros((n_users, N), dtype = np.uint32)
        for user in range(n_users):
            top_n = self._recommend_user(ratings, user, N)
            recommendation[user] = top_n
            
        return recommendation
    
    def _recommend_user(self, ratings, user, N):
        '''top-N ranked items for given user'''
        scores = self._predict_user(user)
        
        # computing top N items, excluding those a user already liked
        liked = set(ratings[user].indices)
        count = N + len(liked)
        if count < scores.shape[0]:
            ids = np.argpartition(scores, -count)[-count:]
            best_ids = np.argsort(scores[ids])[::-1]
            best = ids[best_ids]
        else:
            best = np.argsort(scores)[::-1]
            
        top_n = list(islice((rec for rec in best if rec not in liked), N))
        return top_n
    
    def get_similar_items(self, N = 5, item_ids = None):
        '''
        return top N similar items for particular item id, cosine distance
        is used as distance metric
        '''
        # cosine distance is proportional to normalized euclidean distance
        normed_factors = normalize(self.item_factors)
        knn = NearestNeighbors(n_neighbors = N + 1, metric = 'euclidean')
        knn.fit(normed_factors)
        
        # returns a distance, index tuple
        if item_ids is not None:
            normed_factors = normed_factors[item_ids]
            
        _, items = knn.kneighbors(normed_factors)
        similar_items = items[:, 1:].astype(np.uint32)
        return similar_items

In [54]:
# choosing random parameters to test the class
bpr_params = {'reg': 0.01,
              'alpha': 0.1,
              'n_iters': 160,
              'hidden': 15,
              'batch_size': 100,
              #'seed' : 1, 
              #'verbose' : True
             }

# training a model
bprmodel = BPRMFClass(**bpr_params)
bprmodel.fit(X_train)


BPRMFClass: 100%|██████████| 160/160 [00:03<00:00, 45.79it/s]


### performance evaluation

In [30]:
# using the ROC-AUC score to evaluate performance of the model between 0.0 and 1.0
def auc_score(model, ratings):
    '''
    computing area under the ROC curve:
    as it computes the auc for every user's prediction and actual interaction
    
    '''
    auc = 0.0
    n_users, n_items = ratings.shape
    for user, row in enumerate(ratings):
        y_pred = model._predict_user(user)
        y_true = np.zeros(n_items)
        y_true[row.indices] = 1
        auc += roc_auc_score(y_true, y_pred)
        
    auc /= n_users
    return auc


In [31]:
# performance scores for training and testing
print(auc_score(bprmodel, X_train))
print(auc_score(bprmodel, X_test))

0.8956550383931248
0.8272192019285816


### recommending items

In [32]:
# getting similar items: top N
bprmodel.get_similar_items(N = 5)

array([[ 256,   49,  274,  236,   99],
       [ 453,    2, 1515, 1483,   54],
       [   1, 1515,  208,  209,  467],
       ...,
       [ 350, 1429,  957,  779, 1448],
       [1486,  415, 1443, 1098, 1479],
       [ 809,  750,  697, 1084, 1353]], dtype=uint32)

In [33]:
# getting top N recommended items
bprmodel.recommend(X_train, N = 5)

array([[ 11, 425, 316, 193, 473],
       [806, 293, 123,   8, 741],
       [301, 744, 743, 683, 312],
       ...,
       [285, 267, 312,  99, 124],
       [301,  11, 180, 172, 744],
       [ 88, 190,  81, 264, 143]], dtype=uint32)

### Hyperparameter tuning

In [ ]:
# pickle/joblib the trained model

# with open('BPRMF_model','wb') as f:
#     pickle.dump(bprmodel, f)

In [ ]:
# load model and make predictions

# with open('BPRMF_model','rb') as fr:
#     model = pickle.load(fr)

In [ ]:
# y = model.predict()
# y

# Log Scale

In [ ]:
# df_cdn_frequency['log_freq']=np.log10(df_cdn_frequency.timestamp)

In [ ]:
# df_cdn_frequency

In [ ]:
# df_cdn_frequency['log_rating']=np.rint(df_cdn_frequency.log_freq).astype(int)

In [ ]:
# df_cdn_frequency